# Lesson 2 : LangGraph Components

In [10]:
# !pip install python-dotenv
# !pip install langchain-openai
# !pip install langchain-community
# !pip install graphviz
# !pip install langchain-community[all]
# !pip install langgraph

### Graphs

- LangGraph is an extension of LangChain that supports graphs.
- Single and multi-agent flows are described and represented as graphs
- allows for extremely controlled flow
- built- in persistence allows for human-in-the-loop workflows

nodes: agents or functions
edges: connect nodes
conditional edges: decisions

agent node, conditional edge
function node, edge

entrypoint: starting node
node

### data/state:
agent state:
- agent state is accessible to all parts of the graph
- it is local to the graph
- can be stored in a persistence layer

Simple:
  class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add

Complex:  
  class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annottated[list[tuple[AgentAction, str]], operator.add]


# Code

Agent State

llm: call_openai (one node)
c_edge:exisst_action (conditional edge)
action: take_action (another node to execute the node)

### State:
class AgentState(TypedDict):  \
  messages: Annotated[list[AnyMessage], operator.add]

In [16]:
import os

In [5]:
from dotenv import load_dotenv
_ = load_dotenv()

In [25]:
# credentials
# oepnai_api_key = os.getenv("OPENAI_API_KEY")
# tavily_api_key = os.getenv("TAVILY_API_KEY")

In [29]:
# reset all those keys
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
tool = TavilySearchResults(tavily_api_key=tavily_api_key)

In [30]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [31]:
# tool = TavilySearchResults(tavily_api_key = TAVILY_API_KEY, max_results=4) #increased number of results
tool = TavilySearchResults(tavily_api_key = TAVILY_API_KEY, max_results=4) #increased number of results

print(type(tool))
print(tool.name)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [32]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [33]:
class Agent:

  ## one function to call openai,
  ## one function to check if there is an action
  ## another function to take the action

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

basically: agent class interacts with a language model and external tools using a state-based flow. It first queries the model for a response, then checks if any tool-related actions are needed and executes them. If a valid tool is called, the agent invokes it and returns the result. It retries if the tool name is invalid. The process loops, alternating between querying the model and taking actions as necessary.

In [34]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [43]:
# !pip install pygraphviz
# !pip install pygraphviz --install-option="--include-path=/usr/local/include/graphviz" --install-option="--library-path=/usr/local/lib/graphviz"
!pip install pygraphviz‑<version>.whl

/bin/bash: line 1: version: No such file or directory


In [50]:
# from IPython.display import Image

# # Image(abot.graph.get_graph().draw_png())

In [45]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_rw0mvpisFLeqDPeMjaGZHpDa', 'type': 'tool_call'}
Back to the model!


In [46]:
result

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rw0mvpisFLeqDPeMjaGZHpDa', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a67d6f5e-bc2e-423f-8faf-4fd627425ee1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_rw0mvpisFLeqDPeMjaGZHpDa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 21, 'total_tokens': 1

In [47]:
result['messages'][-1].content

'The current weather in San Francisco is overcast with a temperature of 60.6°F (15.9°C). The wind speed is 5.4 mph (8.6 kph) coming from the WNW direction. The humidity is at 90%, and the cloud cover is 93%.'

In [48]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_oGF9r7lHNgN0rJ5LRMKxd2tn', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_AxKowcnxJHWSTqAUGrPanJWI', 'type': 'tool_call'}
Back to the model!


In [51]:
result['messages'][-1].content

'The current weather in San Francisco is 60.6°F with overcast conditions and a slight wind of 8.6 kph coming from the west-northwest. The humidity is at 90%.\n\nIn Los Angeles, the current weather is 77.2°F with clear skies and a gentle wind of 10.1 kph blowing from the west-southwest. The humidity in Los Angeles is at 47%.'

In [52]:
# Note, the query was modified to produce more consistent results.
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_BSduIN9VHkkaXTVeC4upeik7', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters state'}, 'id': 'call_xKs4JSakaS82vk8QTNI2Ey8O', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP 2024'}, 'id': 'call_k5mNWhXvlQACVZQKtL0r6GC1', 'type': 'tool_call'}
Back to the model!


In [53]:
print(result['messages'][-1].content)

1. The Kansas City Chiefs won the Super Bowl in 2024.

2. The headquarters of the Kansas City Chiefs is located in Missouri.

3. Missouri's GDP was approximately $397 billion in 2022, and while specific 2024 data isn't provided, it likely experienced growth similar to previous years.


## Put things together

What it doesn mean

In [54]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt)

In [56]:
messages = [HumanMessage(content="What is the weather like in Chengdu? What is the weather like in NYC?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chengdu'}, 'id': 'call_tv5JfCkV3JLzdkmpXVV4SpZg', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in NYC'}, 'id': 'call_hvP18UHKqgmd7c5c5eHToCNl', 'type': 'tool_call'}
Back to the model!


In [57]:
result

{'messages': [HumanMessage(content='What is the weather like in Chengdu? What is the weather like in NYC?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tv5JfCkV3JLzdkmpXVV4SpZg', 'function': {'arguments': '{"query": "weather in Chengdu"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_hvP18UHKqgmd7c5c5eHToCNl', 'function': {'arguments': '{"query": "weather in NYC"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 163, 'total_tokens': 219, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e70d2421-b539-4300-8650-1b60cf52a8c0-0', tool_calls=[{'name': 'tavily_search_resu

In [58]:
result['messages'][-1].content

'The current weather in Chengdu, China is light rain with a temperature of 18.2°C (64.8°F). The wind is blowing at 3.6 kph from the north-northeast direction. The humidity is at 94% with a cloud cover of 75%.\n\nIn New York City, United States, it is currently clear with a temperature of 17.8°C (64.0°F). The wind is blowing at 11.2 kph from the north-northwest direction. The humidity is at 28% with no precipitation.'

In [61]:
messages = [HumanMessage(content="What is the lowest temperature in NYC this year? And what kind of clothes would you recommend for such weather?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'lowest temperature in NYC 2021'}, 'id': 'call_bQT2N2stQXjWuxTAYcgeWi7P', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'lowest temperature in NYC 2021'}, 'id': 'call_r71ZFWLT4URr2sUSf266PdPe', 'type': 'tool_call'}
Back to the model!


In [62]:
result['messages'][-1].content

"The lowest temperature recorded in New York City in 2021 was 14°F. For such cold weather, I would recommend wearing layered clothing to stay warm. Here are some suggestions for clothing in that temperature:\n\n1. **Base Layer**: Start with a thermal or moisture-wicking base layer to keep the body insulated and dry.\n2. **Middle Layer**: Wear a sweater or fleece for added warmth.\n3. **Outer Layer**: A windproof and waterproof jacket will protect you from the cold and elements.\n4. **Bottoms**: Opt for thermal leggings or pants to keep your legs warm.\n5. **Accessories**: Don't forget gloves, a hat, and a scarf to cover exposed areas and retain body heat.\n\nLayering is key to staying warm in cold temperatures as it traps heat close to the body."

In [64]:
query = "What are some good ideas to stay warm in NYC?"
messages = [HumanMessage(content=query)]
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [65]:
print(result['messages'][-1].content)

Staying warm in New York City, especially during the cold winter months, can be challenging. Here are some effective ideas to help you stay warm:

1. **Layer Up**: Wear multiple layers of clothing. Start with a moisture-wicking base layer, add a warm middle layer like a sweater or fleece, and top it off with a windproof and waterproof outer layer.

2. **Invest in a Good Coat**: A high-quality, insulated winter coat or parka can make a significant difference in keeping you warm.

3. **Warm Accessories**: Don't forget hats, scarves, gloves, and thermal socks. These items help retain body heat and protect extremities from the cold.

4. **Insulated Footwear**: Wear waterproof and insulated boots to keep your feet warm and dry, especially if there's snow on the ground.

5. **Use Hand and Foot Warmers**: These small, disposable packets can be placed in gloves and boots for additional warmth.

6. **Stay Indoors When Possible**: Spend more time indoors at cafes, museums, or other indoor attrac